# Phase 1: Data Analysis

This notebook analyzes the structure of `Data/Molise.dta` to understand:
- Available variables and their types
- Temporal coverage
- Region distribution
- Worker classification fields
- Outcome variables
- Data quality issues


In [87]:
import pandas as pd
import numpy as np
import pyreadstat
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Set up paths
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "Data"
OUT_DIR = BASE_DIR / "out"
DERIVED_DIR = BASE_DIR / "data" / "derived"

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Data file exists: {(DATA_DIR / 'Molise.dta').exists()}")


Base directory: /Users/vittoriogaravelli/GitHub/GitHub/ERMDA-30464_GP
Data directory: /Users/vittoriogaravelli/GitHub/GitHub/ERMDA-30464_GP/Data
Data file exists: True


In [88]:
# Load the Stata files (Molise, Basilicata, and Puglia if available)
molise_file = DATA_DIR / "Molise.dta"
basilicata_file = DATA_DIR / "Basilicata.dta"
puglia_file = DATA_DIR / "Puglia.dta"

region_specs = [
    ("Molise", molise_file, "12"),
    ("Basilicata", basilicata_file, "2"),
    ("Puglia", puglia_file, "16"),
]

dfs = []
meta = None

for region_name, path, region_code in region_specs:
    if not path.exists():
        print(f"Note: {path} not found - skipping {region_name}")
        continue

    try:
        df_region, meta_region = pyreadstat.read_dta(path)
        print(f"Loaded {region_name}: {len(df_region):,} rows using pyreadstat")
    except Exception as e:
        print(f"pyreadstat failed for {region_name}: {e}, trying pandas...")
        df_region = pd.read_stata(path)
        meta_region = None
        print(f"Loaded {region_name}: {len(df_region):,} rows using pandas")

    if meta is None and meta_region is not None:
        meta = meta_region

    df_region["region_res"] = region_code
    dfs.append(df_region)

if not dfs:
    raise FileNotFoundError("No data files found")

if len(dfs) > 1:
    df = pd.concat(dfs, ignore_index=True)
    print(f"\nCombined dataset: {len(df):,} rows")
else:
    df = dfs[0]
    print(f"\nSingle dataset: {len(df):,} rows")

print(f"\nDataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


Loaded Molise: 298,889 rows using pyreadstat
Loaded Basilicata: 555,155 rows using pyreadstat
Note: /Users/vittoriogaravelli/GitHub/GitHub/ERMDA-30464_GP/Data/Puglia.dta not found - skipping Puglia

Combined dataset: 854,044 rows

Dataset shape: (854044, 24)
Memory usage: 541.06 MB


In [89]:
# Basic info about the dataset
print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
print(f"\nShape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nData types:")
print(df.dtypes)


DATASET OVERVIEW

Shape: 854,044 rows × 24 columns

Column names:
['id_worker', 'year', 'year_birth', 'year_death', 'gender', 'region_res', 'year_pension', 'type', 'id_firm', 'date_start', 'date_end', 'working_weeks', 'wage', 'reason_end', 'part_time', 'part_time_fraction', 'contract_type', 'occupation', 'incentive_policy', 'firm_dimension', 'sector_2d', 'sector_12cat', 'firm_position', 'id_firm_parent']

Data types:
id_worker               int64
year                    int64
year_birth              int64
year_death             object
gender                  int64
region_res             object
year_pension           object
type                    int64
id_firm                object
date_start             object
date_end               object
working_weeks          object
wage                   object
reason_end             object
part_time              object
part_time_fraction    float64
contract_type          object
occupation             object
incentive_policy       object
firm_dime

In [90]:
# Display first few rows
print("=" * 80)
print("SAMPLE DATA (first 5 rows)")
print("=" * 80)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
print(df.head())


SAMPLE DATA (first 5 rows)
   id_worker  year  year_birth year_death  gender region_res year_pension  \
0       3052  2005        1981        NaN       0         12          NaN   
1       3052  2002        1981        NaN       0         12          NaN   
2       3052  2017        1981        NaN       0         12          NaN   
3       3052  2016        1981        NaN       0         12          NaN   
4       3052  2008        1981        NaN       0         12          NaN   

   type  id_firm  date_start date_end working_weeks   wage reason_end  \
0     1  3583834  2005-12-27      NaN             1    300        NaN   
1     4      NaN         NaN      NaN           NaN    NaN        NaN   
2     1  2226770         NaN      NaN            52  19100        NaN   
3     1  2226770         NaN      NaN            52  18100        NaN   
4     1  5743594  2008-01-24      NaN            50  18700        NaN   

  part_time  part_time_fraction contract_type occupation incentive_poli

In [ ]:
# Filter dataset for Basilicata, Molise, and Puglia with selected columns
columns_to_keep = ["id_worker", "year", "type", "wage", "contract_type", "sector_12cat", "region_res"]

if "region_res" not in df.columns:
    raise KeyError("Column 'region_res' not found in dataframe")

region_filters = {
    "basilicata": "2",
    "molise": "12",
    "puglia": "16",
}

filtered_subsets = {}
region_series = df["region_res"].astype(str)

for region_name, region_code in region_filters.items():
    mask = region_series == region_code
    if mask.any():
        subset = df.loc[mask, columns_to_keep].copy()
        filtered_subsets[region_name] = subset
        print(f"{region_name.title()} filtered shape: {subset.shape}")
    else:
        filtered_subsets[region_name] = pd.DataFrame(columns=columns_to_keep)
        print(f"Warning: no rows found for region code {region_code} ({region_name})")

basilicata_filtered = filtered_subsets["basilicata"]
molise_filtered = filtered_subsets["molise"]
puglia_filtered = filtered_subsets["puglia"]


KeyError: "['sector_firm'] not in index"

In [ ]:
# Verify that sector_12cat column is present in filtered datasets
print("=" * 80)
print("VERIFICATION: sector_12cat column presence")
print("=" * 80)
for region_name, subset in {
    "basilicata": basilicata_filtered,
    "molise": molise_filtered,
    "puglia": puglia_filtered,
}.items():
    if subset.empty:
        print(f"{region_name.title()}: No data")
    else:
        has_sector_12cat = "sector_12cat" in subset.columns
        print(f"{region_name.title()}: sector_12cat present = {has_sector_12cat}")
        if has_sector_12cat:
            print(f"  Columns: {list(subset.columns)}")
            print(f"  sector_12cat non-null count: {subset['sector_12cat'].notna().sum():,} / {len(subset):,}")
            # Show unique values
            unique_vals = subset['sector_12cat'].dropna().unique()
            print(f"  Unique sector_12cat values: {sorted([str(v) for v in unique_vals])}")
        print()


VERIFICATION: sector_12cat column presence
Basilicata: sector_12cat present = True
  Columns: ['id_worker', 'year', 'type', 'wage', 'contract_type', 'sector_12cat', 'region_res']
  sector_12cat non-null count: 193,599 / 555,155
  Unique sector_12cat values: ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']

Molise: sector_12cat present = True
  Columns: ['id_worker', 'year', 'type', 'wage', 'contract_type', 'sector_12cat', 'region_res']
  sector_12cat non-null count: 104,316 / 298,889
  Unique sector_12cat values: ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']

Puglia: No data


In [ ]:
# Inspect mapping of 'type' to identify non-employed code
print("Type value counts (overall):")
print(df["type"].value_counts().sort_index())

if 'type' in meta.variable_value_labels:
    print("\nType labels from metadata:")
    print(meta.variable_value_labels['type'])



Type value counts (overall):
type
1    354897
2     52545
3     48611
4    397991
Name: count, dtype: int64

Type labels from metadata:
{1: 'Private employees', 2: 'Public employees', 3: 'Self-employed', 4: 'Non employed'}


In [ ]:
# Create binary employed indicator
regional_sets = {
    "basilicata": basilicata_filtered,
    "molise": molise_filtered,
    "puglia": puglia_filtered,
}

for region_name, subset in regional_sets.items():
    if subset.empty:
        print(f"{region_name} employed share: n/a (no data)")
        continue
    wage_positive = pd.to_numeric(subset["wage"], errors="coerce").fillna(0) > 0
    type_non_employed = subset["type"].eq(4)
    subset["employed"] = (wage_positive | ~type_non_employed).astype("Int8")
    print(f"{region_name} employed share: {subset['employed'].mean():.4f}")


basilicata employed share: 0.5292
molise employed share: 0.5428
puglia employed share: n/a (no data)


In [ ]:
# Retain only workers employed in 1999, 2000, or 2001
COHORT_YEARS = {1999, 2000, 2001}
filtered_results = {}

for region_name, subset in {
    "basilicata": basilicata_filtered,
    "molise": molise_filtered,
    "puglia": puglia_filtered,
}.items():
    if subset.empty:
        filtered_results[region_name] = subset
        print(f"{region_name}: retained 0 rows (no data)")
        continue

    eligible_workers = subset.loc[
        subset["year"].isin(COHORT_YEARS) & subset["employed"].eq(1),
        "id_worker"
    ].unique()
    filtered_subset = subset[subset["id_worker"].isin(eligible_workers)].copy()
    filtered_results[region_name] = filtered_subset
    print(
        f"{region_name}: retained {len(filtered_subset):,} rows for"
        f" {len(eligible_workers):,} eligible workers"
    )

basilicata_filtered = filtered_results["basilicata"]
molise_filtered = filtered_results["molise"]
puglia_filtered = filtered_results["puglia"]



basilicata: retained 292,964 rows for 9,875 eligible workers
molise: retained 160,799 rows for 5,431 eligible workers
puglia: retained 0 rows (no data)


In [ ]:
YEAR_WINDOW = range(1999, 2007)

restricted_sets = {}
for region_name, subset in {
    "Basilicata": basilicata_filtered,
    "Molise": molise_filtered,
    "Puglia": puglia_filtered,
}.items():
    restricted = subset[subset["year"].isin(YEAR_WINDOW)].copy()
    restricted_sets[region_name.lower()] = restricted

print("After year restriction:")
for region_name, subset in restricted_sets.items():
    print(f"  {region_name.title()}: {len(subset):,} rows")

basilicata_filtered = restricted_sets["basilicata"]
molise_filtered = restricted_sets["molise"]
puglia_filtered = restricted_sets["puglia"]



After year restriction:
  Basilicata: 75,905 rows
  Molise: 41,610 rows
  Puglia: 0 rows


In [ ]:
# Create sector_12cat_cont: assign each worker the latest seen sector_12cat value up to each year
# This handles panel data gaps by forward-filling the most recent sector for each worker
print("=" * 80)
print("CREATING sector_12cat_cont COLUMN (latest sector_12cat per worker, per year)")
print("=" * 80)

for region_name, subset in {
    "Basilicata": basilicata_filtered,
    "Molise": molise_filtered,
    "Puglia": puglia_filtered,
}.items():
    if subset.empty:
        print(f"\n{region_name}: No data - skipping")
        continue
    
    # Sort by worker and year to ensure chronological order
    subset_sorted = subset.sort_values(["id_worker", "year"]).copy()
    
    # For each worker, forward-fill the sector_12cat (carry forward latest known value up to each year)
    # This means if a worker was in "2" in 2000, and we don't have data for 2001,
    # we assign "2" to them in 2001 (using the latest known value up to that year)
    subset_sorted["sector_12cat_cont"] = subset_sorted.groupby("id_worker")["sector_12cat"].ffill()
    
    # Update the original filtered dataframe
    if region_name == "Basilicata":
        basilicata_filtered = subset_sorted.sort_index()
    elif region_name == "Molise":
        molise_filtered = subset_sorted.sort_index()
    elif region_name == "Puglia":
        puglia_filtered = subset_sorted.sort_index()
    
    # Print overall diagnostics
    print(f"\n{region_name}:")
    print(f"  Total rows: {len(subset_sorted):,}")
    print(f"  Rows with original sector_12cat: {subset_sorted['sector_12cat'].notna().sum():,}")
    print(f"  Rows with sector_12cat_cont: {subset_sorted['sector_12cat_cont'].notna().sum():,}")
    print(f"  Rows filled (had NaN, now have value): {(subset_sorted['sector_12cat_cont'].notna() & subset_sorted['sector_12cat'].isna()).sum():,}")
    
    # Print year-by-year breakdown
    print(f"\n  Year-by-year breakdown:")
    for year in sorted(subset_sorted["year"].unique()):
        year_data = subset_sorted[subset_sorted["year"] == year]
        total_rows = len(year_data)
        with_original = year_data["sector_12cat"].notna().sum()
        with_cont = year_data["sector_12cat_cont"].notna().sum()
        filled = (year_data["sector_12cat_cont"].notna() & year_data["sector_12cat"].isna()).sum()
        print(f"    {year}: {total_rows:,} rows | {with_original:,} with original sector_12cat | {with_cont:,} with sector_12cat_cont | {filled:,} filled")


CREATING sector_12cat_cont COLUMN (latest sector_12cat per worker, per year)

Basilicata:
  Total rows: 75,905
  Rows with original sector_12cat: 43,851
  Rows with sector_12cat_cont: 55,970
  Rows filled (had NaN, now have value): 12,119

  Year-by-year breakdown:
    1999: 9,732 rows | 5,412 with original sector_12cat | 5,412 with sector_12cat_cont | 0 filled
    2000: 9,753 rows | 5,548 with original sector_12cat | 6,429 with sector_12cat_cont | 881 filled
    2001: 9,706 rows | 6,050 with original sector_12cat | 7,299 with sector_12cat_cont | 1,249 filled
    2002: 9,590 rows | 5,648 with original sector_12cat | 7,321 with sector_12cat_cont | 1,673 filled
    2003: 9,475 rows | 5,355 with original sector_12cat | 7,278 with sector_12cat_cont | 1,923 filled
    2004: 9,350 rows | 5,132 with original sector_12cat | 7,237 with sector_12cat_cont | 2,105 filled
    2005: 9,216 rows | 5,435 with original sector_12cat | 7,535 with sector_12cat_cont | 2,100 filled
    2006: 9,083 rows | 5,2

In [ ]:
# Count how many people work in sector_12cat == "2" each year
print("=" * 80)
print("PEOPLE WORKING IN SECTOR_12CAT == '2' BY YEAR")
print("=" * 80)

for region_name, subset in {
    "Basilicata": basilicata_filtered,
    "Molise": molise_filtered,
    "Puglia": puglia_filtered,
}.items():
    if subset.empty:
        print(f"\n{region_name}: No data")
        continue
    
    # Filter for sector_12cat == 2 (handle both numeric and string)
    # Convert to string for comparison to handle both numeric 2 and string "2"
    sector_2_mask = subset["sector_12cat"].astype(str) == "2"
    
    if not sector_2_mask.any():
        print(f"\n{region_name}: No workers in sector_12cat == 2")
        continue
    
    # Count unique workers per year in sector 2
    sector_2_data = subset[sector_2_mask].copy()
    workers_by_year = sector_2_data.groupby("year")["id_worker"].nunique().sort_index()
    
    print(f"\n{region_name}:")
    print(f"  Total rows with sector_12cat == '2': {len(sector_2_data):,}")
    print(f"  Unique workers in sector 2 by year:")
    for year, count in workers_by_year.items():
        print(f"    {year}: {count:,} unique workers")
    
    # Also show total count across all years
    total_unique_workers = sector_2_data["id_worker"].nunique()
    print(f"  Total unique workers (across all years): {total_unique_workers:,}")


PEOPLE WORKING IN SECTOR_12CAT == '2' BY YEAR

Basilicata: No workers in sector_12cat == '2'

Molise: No workers in sector_12cat == '2'

Puglia: No data


In [ ]:
# Compare sector_12cat vs sector_12cat_cont for sector 2
print("=" * 80)
print("COMPARISON: sector_12cat vs sector_12cat_cont for '2'")
print("=" * 80)

for region_name, subset in {
    "Basilicata": basilicata_filtered,
    "Molise": molise_filtered,
    "Puglia": puglia_filtered,
}.items():
    if subset.empty:
        print(f"\n{region_name}: No data")
        continue
    
    # Count using original sector_12cat (handle both numeric and string)
    sector_2_original = (subset["sector_12cat"].astype(str) == "2").sum()
    workers_original = subset[subset["sector_12cat"].astype(str) == "2"].groupby("year")["id_worker"].nunique().sort_index()
    
    # Count using sector_12cat_cont (handle both numeric and string)
    sector_2_cont = (subset["sector_12cat_cont"].astype(str) == "2").sum()
    workers_cont = subset[subset["sector_12cat_cont"].astype(str) == "2"].groupby("year")["id_worker"].nunique().sort_index()
    
    print(f"\n{region_name}:")
    print(f"  Rows with sector_12cat == '2': {sector_2_original:,}")
    print(f"  Rows with sector_12cat_cont == '2': {sector_2_cont:,}")
    print(f"  Difference (filled): {sector_2_cont - sector_2_original:,} additional rows")
    print(f"\n  Unique workers by year (using sector_12cat_cont):")
    for year, count in workers_cont.items():
        orig_count = workers_original.get(year, 0)
        diff = count - orig_count
        print(f"    {year}: {count:,} unique workers (original: {orig_count:,}, +{diff:,})")


COMPARISON: sector_12cat vs sector_12cat_cont for '2'

Basilicata:
  Rows with sector_12cat == '2': 0
  Rows with sector_12cat_cont == '2': 0
  Difference (filled): 0 additional rows

  Unique workers by year (using sector_12cat_cont):

Molise:
  Rows with sector_12cat == '2': 0
  Rows with sector_12cat_cont == '2': 0
  Difference (filled): 0 additional rows

  Unique workers by year (using sector_12cat_cont):

Puglia: No data


In [ ]:
# Analyze transitions and persistence in sector 2 using sector_12cat_cont
print("=" * 80)
print("TRANSITIONS AND PERSISTENCE IN '2' (using sector_12cat_cont)")
print("=" * 80)

for region_name, subset in {
    "Basilicata": basilicata_filtered,
    "Molise": molise_filtered,
    "Puglia": puglia_filtered,
}.items():
    if subset.empty:
        print(f"\n{region_name}: No data")
        continue
    
    # Filter for sector_12cat_cont == 2 (handle both numeric and string)
    sector_2_data = subset[subset["sector_12cat_cont"].astype(str) == "2"].copy()
    
    if len(sector_2_data) == 0:
        print(f"\n{region_name}: No workers in sector_12cat_cont == 2")
        continue
    
    # Get unique workers by year
    workers_by_year = {}
    for year in sorted(sector_2_data["year"].unique()):
        workers_by_year[year] = set(sector_2_data[sector_2_data["year"] == year]["id_worker"].unique())
    
    print(f"\n{region_name}:")
    print(f"  Analysis of workers in sector_12cat_cont == '2':")
    
    # Analyze year by year
    years = sorted(workers_by_year.keys())
    for i, year in enumerate(years):
        current_workers = workers_by_year[year]
        print(f"\n    {year}: {len(current_workers):,} unique workers")
        
        if i > 0:
            prev_year = years[i-1]
            prev_workers = workers_by_year[prev_year]
            
            # Workers who continued from previous year
            continuing = current_workers & prev_workers
            # New entrants (in current year but not in previous)
            entrants = current_workers - prev_workers
            # Exits (in previous year but not in current)
            exits = prev_workers - current_workers
            
            print(f"      Continuing from {prev_year}: {len(continuing):,} workers")
            print(f"      New entrants: {len(entrants):,} workers")
            if len(entrants) > 0 and len(entrants) <= 10:
                print(f"        Worker IDs: {sorted(list(entrants))}")
            print(f"      Exits (left sector 2): {len(exits):,} workers")
            if len(exits) > 0 and len(exits) <= 10:
                print(f"        Worker IDs: {sorted(list(exits))}")
    
    # Find workers who appear in multiple consecutive years
    print(f"\n  Persistence analysis:")
    worker_years = sector_2_data.groupby("id_worker")["year"].apply(lambda x: sorted(set(x))).to_dict()
    
    # Count workers by how many years they appear
    years_count = {}
    for worker, years_list in worker_years.items():
        num_years = len(years_list)
        if num_years not in years_count:
            years_count[num_years] = []
        years_count[num_years].append((worker, years_list))
    
    for num_years in sorted(years_count.keys(), reverse=True):
        workers_list = years_count[num_years]
        print(f"    Workers appearing in {num_years} year(s): {len(workers_list):,}")
        if len(workers_list) <= 5:
            for worker, years_list in workers_list:
                print(f"      Worker {worker}: years {years_list}")
    
    # Show if same worker appears across multiple years (like 1999-2002)
    print(f"\n  Workers appearing in consecutive years:")
    consecutive_workers = []
    for worker, years_list in worker_years.items():
        if len(years_list) > 1:
            # Check if years are consecutive
            years_sorted = sorted(years_list)
            is_consecutive = all(years_sorted[i] == years_sorted[i-1] + 1 for i in range(1, len(years_sorted)))
            if is_consecutive:
                consecutive_workers.append((worker, years_list))
    
    consecutive_workers.sort(key=lambda x: len(x[1]), reverse=True)
    for worker, years_list in consecutive_workers[:10]:  # Show top 10
        print(f"    Worker {worker}: {years_list[0]}-{years_list[-1]} ({len(years_list)} years)")


TRANSITIONS AND PERSISTENCE IN '2' (using sector_12cat_cont)

Basilicata: No workers in sector_12cat_cont == '2'

Molise: No workers in sector_12cat_cont == '2'

Puglia: No data


In [ ]:
# Wage range diagnostics
print("Wage min/max by region:")
for region_name, subset in {
    "Basilicata": basilicata_filtered,
    "Molise": molise_filtered,
    "Puglia": puglia_filtered,
}.items():
    if subset.empty or "wage" not in subset.columns:
        print(f"  {region_name}: n/a")
        continue
    wages = pd.to_numeric(subset["wage"], errors="coerce")
    print(
        f"  {region_name}: min={wages.min(skipna=True):,.2f},"
        f" max={wages.max(skipna=True):,.2f}"
    )



Wage min/max by region:
  Basilicata: min=0.00, max=163,300.00
  Molise: min=0.00, max=175,300.00
  Puglia: n/a


In [ ]:
# Diagnostic: Check wage values before creating income_category
print("Wage diagnostics before income_category creation:")
for region_name, subset in {
    "Basilicata": basilicata_filtered,
    "Molise": molise_filtered,
    "Puglia": puglia_filtered,
}.items():
    if subset.empty or "wage" not in subset.columns:
        print(f"  {region_name}: n/a")
        continue
    wages = pd.to_numeric(subset["wage"], errors="coerce")
    print(f"  {region_name}:")
    print(f"    Total rows: {len(subset):,}")
    print(f"    Non-null wages: {wages.notna().sum():,}")
    print(f"    Null wages: {wages.isna().sum():,}")
    print(f"    Wages > 0: {(wages > 0).sum():,}")
    print(f"    Wages == 0: {(wages == 0).sum():,}")
    if wages.notna().any():
        print(f"    Min wage: {wages.min():,.2f}")
        print(f"    Max wage: {wages.max():,.2f}")
        print(f"    Mean wage: {wages.mean():,.2f}")
    print()


Wage diagnostics before income_category creation:
  Basilicata:
    Total rows: 75,905
    Non-null wages: 62,426
    Null wages: 13,479
    Wages > 0: 62,343
    Wages == 0: 83
    Min wage: 0.00
    Max wage: 163,300.00
    Mean wage: 15,826.99

  Molise:
    Total rows: 41,610
    Non-null wages: 34,302
    Null wages: 7,308
    Wages > 0: 34,210
    Wages == 0: 92
    Min wage: 0.00
    Max wage: 175,300.00
    Mean wage: 16,172.84

  Puglia: n/a


In [ ]:
# Add income_category column based on wage
for region_name, subset in {
    "Basilicata": basilicata_filtered,
    "Molise": molise_filtered,
    "Puglia": puglia_filtered,
}.items():
    if subset.empty or "wage" not in subset.columns:
        print(f"  {region_name}: n/a (no data or no wage column)")
        continue
    
    # Convert wage to numeric
    wages = pd.to_numeric(subset["wage"], errors="coerce")
    
    # Initialize income_category column with NaN
    income_cat = pd.Series(index=subset.index, dtype="Int8")
    
    # Only assign categories for non-NaN wages
    valid_mask = wages.notna()
    if valid_mask.any():
        valid_wages = wages[valid_mask]
        
        # Create income category column (using numeric codes: 1=low, 2=medium, 3=high)
        conditions = [
            valid_wages < 28000,
            (valid_wages >= 28000) & (valid_wages <= 50000),
            valid_wages > 50000
        ]
        choices = [1, 2, 3]  # 1=low income, 2=medium, 3=high
        
        # Assign categories only for valid wages
        income_cat[valid_mask] = np.select(conditions, choices, default=None)
    
    # Assign to the subset DataFrame
    subset["income_category"] = income_cat.astype("Int8")
    
    print(f"  {region_name}: income_category distribution:")
    print(subset["income_category"].value_counts().sort_index())
    print(f"  {region_name}: NaN count in income_category: {subset['income_category'].isna().sum():,}")
    print()


  Basilicata: income_category distribution:
income_category
1    56580
2     4713
3     1133
Name: count, dtype: Int64
  Basilicata: NaN count in income_category: 13,479

  Molise: income_category distribution:
income_category
1    30860
2     2836
3      606
Name: count, dtype: Int64
  Molise: NaN count in income_category: 7,308

  Puglia: n/a (no data or no wage column)


In [ ]:
# Save filtered datasets
DERIVED_DIR.mkdir(parents=True, exist_ok=True)

output_specs = {
    "basilicata": (basilicata_filtered, DERIVED_DIR / "basilicata_filtered.dta"),
    "molise": (molise_filtered, DERIVED_DIR / "molise_filtered.dta"),
    "puglia": (puglia_filtered, DERIVED_DIR / "puglia_filtered.dta"),
}

for region_name, (subset, path) in output_specs.items():
    if subset.empty:
        print(f"Skipping {region_name} export (no data)")
        continue
    subset.convert_dtypes().to_stata(path, write_index=False, version=118)
    print(f"Saved {region_name.title()} filtered data to {path}")


Saved Basilicata filtered data to /Users/vittoriogaravelli/GitHub/GitHub/ERMDA-30464_GP/data/derived/basilicata_filtered.dta
Saved Molise filtered data to /Users/vittoriogaravelli/GitHub/GitHub/ERMDA-30464_GP/data/derived/molise_filtered.dta
Skipping puglia export (no data)
